# Python pandas tricks

Notebook to go through examples of top 25 pandas tricks in the following video: https://www.dataschool.io/python-pandas-tricks/

In [1]:
import pandas as pd
import numpy as np

## 1 - Show installed versions

In [2]:
pd.__version__

'0.22.0'

#### Show pandas dependecies versions

In [3]:
pd.show_versions()


INSTALLED VERSIONS
------------------
commit: None
python: 3.6.4.final.0
python-bits: 64
OS: Darwin
OS-release: 16.7.0
machine: x86_64
processor: i386
byteorder: little
LC_ALL: None
LANG: en_US.UTF-8
LOCALE: en_US.UTF-8

pandas: 0.22.0
pytest: 3.3.2
pip: 10.0.1
setuptools: 38.4.0
Cython: 0.27.3
numpy: 1.14.0
scipy: 1.0.0
pyarrow: 0.9.0.post1
xarray: None
IPython: 6.2.1
sphinx: 1.6.6
patsy: 0.5.0
dateutil: 2.6.1
pytz: 2017.3
blosc: None
bottleneck: 1.2.1
tables: 3.4.2
numexpr: 2.6.4
feather: None
matplotlib: 2.1.2
openpyxl: 2.4.10
xlrd: 1.1.0
xlwt: 1.2.0
xlsxwriter: 1.0.2
lxml: 4.1.1
bs4: 4.6.0
html5lib: 1.0.1
sqlalchemy: 1.2.1
pymysql: None
psycopg2: 2.8.1 (dt dec pq3 ext lo64)
jinja2: 2.10
s3fs: None
fastparquet: None
pandas_gbq: None
pandas_datareader: None


## 2 - Create an example DataFrame

In [6]:
# from a dict
df = pd.DataFrame({'col one': [100,200], 'col two': [300,400]})
df

,col one,col two
0,100,300
1,200,400


In [8]:
pd.DataFrame(np.random.rand(4,8), columns = list('abcdefgh'))

,a,b,c,d,e,f,g,h
0,0.620423,0.342195,0.632548,0.196611,0.054357,0.804252,0.602507,0.244342
1,0.786716,0.276336,0.041872,0.156508,0.638271,0.059495,0.609333,0.121332
2,0.467013,0.495198,0.350927,0.346083,0.610613,0.188152,0.938695,0.158600
3,0.350827,0.765888,0.002043,0.377600,0.603116,0.852209,0.293429,0.419574


## 3 - Rename columns

In [9]:
df = df.rename({'col one':'col_one', 'col two':'col_two'}, axis='columns')

In [10]:
df.columns = ['col_one', 'col_two']

In [11]:
df.columns = df.columns.str.replace(' ', '_')

In [12]:
df.add_prefix('col_prefix_')

,col_prefix_col_one,col_prefix_col_two
0,100,300
1,200,400


## 4 - Reverse row order

In [13]:
df.loc[::-1].head()

,col_one,col_two
1,200,400
0,100,300


In [14]:
df.loc[::-1].reset_index(drop=True).head()

,col_one,col_two
0,200,400
1,100,300


## 5 - Reverse column order

In [15]:
df.loc[:,::-1].head()

,col_two,col_one
0,300,100
1,400,200


## 6 - Select columns by data type

In [17]:
df.select_dtypes(include='number').head()

,col_one,col_two
0,100,300
1,200,400


In [18]:
df.select_dtypes(exclude='number').head()

""
0
1


## 7 - Convert strings to numbers

In [20]:
df = pd.DataFrame({'col_one':['1.1', '2.2', '3.3'],
                  'col_two':['4.4', '5.5', '6.6'],
                  'col_three':['7.7', '8.8', '-']})
df

,col_one,col_three,col_two
0,1.1,7.7,4.4
1,2.2,8.8,5.5
2,3.3,-,6.6


In [21]:
pd.to_numeric(df['col_three'], errors='coerce').fillna(0)

0    7.7
1    8.8
2    0.0
Name: col_three, dtype: float64

In [22]:
df = df.apply(pd.to_numeric, errors='coerce').fillna(0)

In [23]:
df

,col_one,col_three,col_two
0,1.1,7.7,4.4
1,2.2,8.8,5.5
2,3.3,0.0,6.6


## 8 - Reduce DataFrame size

In [26]:
drinks_df = pd.read_csv('http://bit.ly/drinksbycountry')
drinks_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 6 columns):
country                         193 non-null object
beer_servings                   193 non-null int64
spirit_servings                 193 non-null int64
wine_servings                   193 non-null int64
total_litres_of_pure_alcohol    193 non-null float64
continent                       193 non-null object
dtypes: float64(1), int64(3), object(2)
memory usage: 30.4 KB


#### Only read in the columns that are necessary

In [28]:
cols_to_read_in = ['beer_servings', 'continent']
small_drinks_df = pd.read_csv('http://bit.ly/drinksbycountry', usecols=cols_to_read_in)
small_drinks_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 2 columns):
beer_servings    193 non-null int64
continent        193 non-null object
dtypes: int64(1), object(1)
memory usage: 13.6 KB


#### Specify categorical features on read

In [29]:
dtypes={'continent':'category'}
smaller_drinks_df = pd.read_csv('http://bit.ly/drinksbycountry', usecols=cols_to_read_in, dtype=dtypes)
smaller_drinks_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 2 columns):
beer_servings    193 non-null int64
continent        193 non-null category
dtypes: category(1), int64(1)
memory usage: 2.3 KB


In [30]:
smaller_drinks_df.dtypes

beer_servings       int64
continent        category
dtype: object

## 9 - Build a DataFrame from multiple files row-wise

In [ ]:
from glob import glob

In [ ]:
list_of_files = sorted(glob('data/stocks*.csv'))

In [ ]:
pd.concat((pd.read_csv(file) for file in list_of_files),ignore_index=True)

## 10 - Build a DataFrame from multiple files column-wise

In [ ]:
pd.concat((pd.read_csv(file) for file in list_of_files), axis='columns')

## 11 - Build a DataFrame from the clipboard

In [32]:
df = pd.read_clipboard()
df

,saledate,mileage,grade,sale_price
0,7/16/19,50229,4.4,26750
1,7/17/19,37926,4.6,28900
2,7/16/19,85784,3.6,19700
3,7/16/19,40443,3.6,27000
4,7/16/19,18147,3.8,36100


## 12 - Split a DataFrame into two random subsets

In [ ]:
movies1 = movies.sample(frac = 0.75, random_state = 123)

In [ ]:
movies2 = movies.drop(movies1.index)

## 13 - Filter a DataFrame by multiple categories

In [37]:
df[(df['mileage']>30000) & (df['grade'] > 3.6)]

,saledate,mileage,grade,sale_price
0,7/16/19,50229,4.4,26750
1,7/17/19,37926,4.6,28900


In [39]:
df[df['mileage'].isin([18147, 50229])]

,saledate,mileage,grade,sale_price
0,7/16/19,50229,4.4,26750
4,7/16/19,18147,3.8,36100


In [40]:
df[~df['mileage'].isin([18147, 50229])]

,saledate,mileage,grade,sale_price
1,7/17/19,37926,4.6,28900
2,7/16/19,85784,3.6,19700
3,7/16/19,40443,3.6,27000


## 14 - Filter a DataFrame by largest categories

In [ ]:
counts = movies['genre'].value_counts().nlargest(3)

In [ ]:
movies[movies['genre'].isin(counts.nlargest(3).index)]

## 15 - Handle missing values

In [41]:
df.isna().sum()

saledate      0
mileage       0
grade         0
sale_price    0
dtype: int64

In [42]:
df.isna().mean()

saledate      0.0
mileage       0.0
grade         0.0
sale_price    0.0
dtype: float64

In [ ]:
df.dropna(thresh=len(df)*0.9, axis='columns').head()

## 16 - Split a string into multiple columns

In [ ]:
df[['first', 'middle', 'last']] = df['name'].str.split(' ', expand=True)

## 17 - Expand a series of lists into a DataFrame

In [43]:
df_new = df['saledate'].apply(pd.Series)

In [44]:
pd.concat([df, df_new], axis='columns')

,saledate,mileage,grade,sale_price,0
0,7/16/19,50229,4.4,26750,7/16/19
1,7/17/19,37926,4.6,28900,7/17/19
2,7/16/19,85784,3.6,19700,7/16/19
3,7/16/19,40443,3.6,27000,7/16/19
4,7/16/19,18147,3.8,36100,7/16/19


## 18 - Aggregate by multiple functions

In [ ]:
orders.groupby('order_id')['item_price'].agg(['sum', 'count']).head()

## 19 - Combine output of aggregation with DataFrame

In [ ]:
orders['total_price'] = orders.groupby('order_id')['item_price'].transform('sum')

## 20 - Select a slice of rows and columns

In [ ]:
df.loc['min':'max']

## 21 - Reshape a multi-index series

In [ ]:
titanic.groupby(['Sex', 'Pclass'])['survived'].mean().unstack()

## 22 - Create a pivot table

In [ ]:
# margins = True adds row and column totals
titanic.pivot_table(index='Sex', columns='Pclass', values='Survived', aggfunc='mean', 
                   margins=True)

In [ ]:
# cross tab created when count used instead of mean
titanic.pivot_table(index='Sex', columns='Pclass', values='Survived', aggfunc='count', 
                   margins=True)

## 23 - Convert continuous data to categorical

In [46]:
pd.cut(df['mileage'], bins=[0,10000,100000], labels=['small', 'med'])

0    med
1    med
2    med
3    med
4    med
Name: mileage, dtype: category
Categories (2, object): [small < med]

## 24 - Change display options

In [48]:
pd.set_option('display.float_format', '{:.2f}'.format)

In [49]:
df

,saledate,mileage,grade,sale_price
0,7/16/19,50229,4.40,26750
1,7/17/19,37926,4.60,28900
2,7/16/19,85784,3.60,19700
3,7/16/19,40443,3.60,27000
4,7/16/19,18147,3.80,36100


## 25 - Style a DataFrame

In [62]:
format_dict = {'mileage':'{0:20,}'}

In [63]:
df.style.format(format_dict)

,saledate,mileage,grade,sale_price
0,7/16/19,"50,229",4.4,26750
1,7/17/19,"37,926",4.6,28900
2,7/16/19,"85,784",3.6,19700
3,7/16/19,"40,443",3.6,27000
4,7/16/19,"18,147",3.8,36100
